In [1]:
import anndata
import scanpy as sc
import scgen
import pandas as pd  
import numpy as np

font = {'family' : 'Arial',
        'size'   : 14}

Global seed set to 0
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [2]:
train = sc.read("./data/train.h5ad",backup_url='https://drive.google.com/uc?id=1r87vhoLLq6PXAYdmyyd89zG90eJOFYLk')

/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [3]:
print(train)
print(train.obs['condition'].values)

AnnData object with n_obs × n_vars = 16893 × 6998
    obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type'
    var: 'gene_symbol', 'n_cells'
    uns: 'cell_type_colors', 'condition_colors', 'neighbors'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'distances', 'connectivities'
['stimulated', 'stimulated', 'stimulated', 'stimulated', 'stimulated', ..., 'control', 'control', 'control', 'control', 'control']
Length: 16893
Categories (2, object): ['control', 'stimulated']


In [4]:
train[train.obs['condition']=='control'].X

<8007x6998 sparse matrix of type '<class 'numpy.float32'>'
	with 2634770 stored elements in Compressed Sparse Row format>

In [5]:
import random
import numpy as np

data = [1, 2, 3, 4, 5]
sample_size = 3

sample = random.sample(data, k=sample_size)
print(sample)
print(sample[:2])
x=dict.fromkeys(range(3))
x[0]=4
np.array(x.values())


[4, 5, 1]
[4, 5]


array(dict_values([4, None, None]), dtype=object)

In [6]:
import pandas as pd  
import numpy as np
def sample_obs(adata,with_replacement,size_n):

    x = adata.X
    n_rows=x.shape[0]
    
    df = pd.DataFrame({'x':np.arange(n_rows)})
    #shuffle the data 
    x_sample1=df['x'].sample(frac=1, replace=with_replacement).values[:size_n]
    x_sample2=df['x'].sample(frac=1, replace=with_replacement).values[:size_n]
    
    return x[x_sample1,:], x[x_sample2,:]

x,y=sample_obs(train,True,train.shape[0])




In [7]:
from sklearn.metrics import mean_absolute_error
from scipy import stats
from scipy.special import kl_div
from scipy.stats import entropy

def calc_pearson(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    
    #x = np.asarray(np.mean(x, axis=0)).ravel()
    #y = np.asarray(np.mean(y, axis=0)).ravel()
    
    #print("mae:",mean_absolute_error(y,x))
    #if actual value is y and predicted value is x
    mae_obs=dict.fromkeys(range(x.shape[0]))
    div_obs=dict.fromkeys(range(x.shape[0]))
    a=dict.fromkeys(range(x.shape[0]))
    pearson_obs=dict.fromkeys(range(x.shape[0]))
    x=x.toarray()
    y=y.toarray()
  
    for i in range(x.shape[0]):
        #use pd
        df = pd.DataFrame({'x':x[i], 'y':y[i]})
        matrix=df.corr(method='pearson').values
        pearson_obs[i] = matrix[0,1]
        
        #use scipy
        a[i]=stats.pearsonr(x[i], y[i]).statistic
 

    
    pr=list(pearson_obs.values())
    #print('a:',np.mean(list(a.values())))
    
   
    return np.mean(pr)

conditions = {"x": "stimulated", "y": "control"}
print(calc_pearson(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_pearson(train,conditions)

0.3760807457999495


0.43203635829693504

In [8]:
from sklearn.metrics import mean_absolute_error
from scipy import stats
from scipy.special import kl_div
from scipy.stats import entropy

def calc_mae(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    
    #x = np.asarray(np.mean(x, axis=0)).ravel()
    #y = np.asarray(np.mean(y, axis=0)).ravel()
    
    #print("mae:",mean_absolute_error(y,x))
    #if actual value is y and predicted value is x
    mae_obs=dict.fromkeys(range(x.shape[0]))
  
    for i in range(x.shape[0]):
        mae_obs[i]=np.mean(np.abs(y[i] - x[i]))
 
    
    mae=list(mae_obs.values())

   
    return np.mean(mae)

conditions = {"x": "stimulated", "y": "control"}
print(calc_mae(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_mae(train,conditions)

0.06448077


0.061301168

In [64]:
from scipy.special import kl_div
from scipy.stats import entropy 
from scipy.special import rel_entr
from scipy.spatial import distance

def calc_kl(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    x=x.toarray()
    y=y.toarray()
       
    div_obs=dict.fromkeys(range(x.shape[0]))
    a=dict.fromkeys(range(x.shape[0]))
    for i in range(x.shape[0]):
#         if np.all(x[i]==0)==False:
#             print("x:",x[i])
#         if np.all(y[i]==0)==False:
#             print("y:",y[i])
        
        a[i]=distance.jensenshannon(x[i],y[i])
        div_obs[i]=entropy(x[i],y[i])
        
       
    sparse=list(a.values())
    div=list(div_obs.values())
    print(sum(sparse)/len(sparse))
    
   
    return np.mean(sparse)

conditions = {"x": "stimulated", "y": "control"}
print(calc_kl(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_kl(train,conditions)

0.7069505837722218
0.7069505837722204
0.6950556263829044


0.6950556263829042

In [65]:
def calc_euclidean_d(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    x=x.toarray()
    y=y.toarray()
       
    d_obs=dict.fromkeys(range(x.shape[0]))
    a=dict.fromkeys(range(x.shape[0]))
    for i in range(x.shape[0]):
        
        d_obs[i]=np.linalg.norm(x[i]-y[i])
        
    
    distance=list(d_obs.values())

   
    return np.mean(distance)

conditions = {"x": "stimulated", "y": "control"}
print(calc_euclidean_d(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_euclidean_d(train,conditions)

21.382507


19.883062

In [67]:

def calc_r(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    x=x.toarray()
    y=y.toarray()
       
    r_obs=dict.fromkeys(range(x.shape[0]))
    
    for i in range(x.shape[0]):
        m, b, r_value, p_value, std_err = stats.linregress(x[i], y[i])
        r_obs[i]=r_value**2
        
    
    rsquare=list(r_obs.values())

   
    return np.mean(rsquare)

conditions = {"x": "stimulated", "y": "control"}
print(calc_r(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_r(train,conditions)

0.1528904405733022


0.2026452078620873

In [10]:
train.obs.condition.value_counts()

stimulated    8886
control       8007
Name: condition, dtype: int64

In [11]:
from scipy.special import kl_div
from scipy.stats import entropy
def calc_kl(adata, conditions=None):

    real_stim = adata[adata.obs["condition"] == conditions["x"]]
    pred_stim = adata[adata.obs["condition"] == conditions["y"]]
    n=np.minimum(control.shape[0],stim.shape[0])
    
    y,_ = sample_obs(pred_stim,True,n)
    x,_ = sample_obs(real_stim,True,n)
   
    x = np.asarray(np.mean(x, axis=0)).ravel()
    y = np.asarray(np.mean(y, axis=0)).ravel()
    
    divergence=kl_div(x,y)
    print("divergence: ",entropy(x,y))
    m, b, r_value, p_value, std_err = stats.linregress(x, y)
    print("r square:",r_value**2)
    df = pd.DataFrame({'x':x, 'y':y})
    # compute pearson correlation
    r = df['x'].corr(df['y'])
    print("pearson_correlation:",r)
    d=np.linalg.norm(x-y)
    print("euclidean distance: ",d)

    
    return divergence

conditions = {"x": "stimulated", "y": "control"}
print(calc_kl(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_kl(train,conditions)

NameError: name 'control' is not defined

In [ ]:
import pandas as pd  
import numpy as np
def sample(adata,with_replacement):

    x = np.asarray(np.mean(adata.X, axis=0)).ravel()# always compute mean across n_obs?
    
    df = pd.DataFrame({'x':x})
    
    x_sample1=df['x'].sample(frac=0.5, replace=with_replacement, random_state=1)
    x_sample2=df['x'].sample(frac=0.5, replace=with_replacement, random_state=2)
    
    return x_sample1.values, x_sample2.values


In [ ]:
def sample2(adata,with_replacement,condition):
    x_adata = adata[adata.obs["condition"] == conditions["x"]]
    y_adata = adata[adata.obs["condition"] == conditions["y"]]
        
    x = np.asarray(np.mean(x_adata.X, axis=0)).ravel()# always compute mean across n_obs?
    y = np.asarray(np.mean(y_adata.X, axis=0)).ravel()
    df = pd.DataFrame({'x':x,'y':y})
    
    x_sample=df['x'].sample(frac=0.5, replace=with_replacement, random_state=1)
    y_sample=df['y'].sample(frac=0.5, replace=with_replacement, random_state=1)
    
    return x_sample.values, y_sample.values

conditions = {"x": "control", "y": "control"}
x_sample, y_sample=sample2(train,True,conditions)

In [ ]:

sample1, sample2=sample(train,True)
print("sample1: ",sample1)
print("sample2: ",sample2)
print(np.sum(sample1-sample2))

In [ ]:
from sklearn.metrics import mean_absolute_error
def calc_mae(adata, conditions=None):

    real_stim = adata[adata.obs["condition"] == conditions["x"]]
    pred_stim = adata[adata.obs["condition"] == conditions["y"]]
    
    y,_ = sample(pred_stim,True)
    x,_ = sample(real_stim,True)
    print(x)
    
    print("mae:",mean_absolute_error(y,x))
    #if actual value is y and predicted value is x
    mae=np.mean(np.abs(np.array(y) - np.array(x)))
    #statistic, p_value = stats.wilcoxon(x-y)

# Output the test results
#     print("Wilcoxon signed-rank statistic:", statistic)
#     print("p-value:", p_value)
    
    return mae
# cell_types = ["B", "CD14-Mono", "CD4-T", "CD8-T", "DC", "F-Mono", "NK"]
conditions = {"x": "stimulated", "y": "control"}
calc_mae(train,conditions)

conditions = {"x": "control", "y": "control"}
calc_mae(train,conditions)

In [ ]:
from scipy.special import kl_div
from scipy.stats import entropy
from scipy import stats
def calc_kl(adata, conditions=None):

    real_stim = adata[adata.obs["condition"] == conditions["x"]]
    pred_stim = adata[adata.obs["condition"] == conditions["y"]]
    
    y,_ = sample(pred_stim,True)
    x,_ = sample(real_stim,True)
    
    m, b, r_value, p_value, std_err = stats.linregress(x, y)
   
    
    divergence=kl_div(x,y)
    print(entropy(x,y))
    print("r square:",r_value**2)
    
    return divergence

conditions = {"x": "stimulated", "y": "control"}
calc_kl(train,conditions)

conditions = {"x": "control", "y": "control"}
calc_kl(train,conditions)

In [ ]:
conditions = {"x": "control", "y": "control"}
calc_mae(train,conditions=conditions)